In [4]:
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
service_obj = Service("C:\\Users\\rashp\\anaconda3\\lib\\site-packages\\chromedriver_binary\\chromedriver.exe")


browser = webdriver.Chrome(service=service_obj) 
df = pd.read_excel('Input_blackcoffer.xlsx')

In [5]:
# Func to extract data(text) 'only text' from the given links(n no. of links) in the datafile 'Input.xlsx' 


for i in tqdm(range(len(df))):
    link = df['URL'][i]       #URL is the column name having links 
    browser.get(link)         #now browser open the link
    time.sleep(1)             #now wait for 1 sec so that page load fully
    soup = BeautifulSoup(browser.page_source , 'html.parser')      #soup now have all the data of that page and pass html parser  
    try:
        d = (soup.find('div' , class_ = 'td-pb-span8 td-main-content').find('div' , class_ = 'td-post-content').text.strip())
        d = d.replace('\n\n', ' ')
        d = d.replace('\n' , ' ')                        # it fetches the data only text data and replaces '\n', '\n\n' with ' space' to make corps data more readable
        fd = open(str(df['URL_ID'][i]), 'w')             # now file is created and open in a write format
        fd.write(d)                                      # and write the content of 'd' in the file
        fd.close()                                       # now close the file..imp to close  
    except:
        pass
                                      
        
        # and now this process occur for each n every link you have in ur Input file

100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [06:09<00:00,  3.24s/it]


In [2]:
!pip install selenium

                                              0.0/6.7 MB ? eta -:--:--
                                              0.0/6.7 MB ? eta -:--:--
                                              0.1/6.7 MB 825.8 kB/s eta 0:00:09
                                              0.1/6.7 MB 901.1 kB/s eta 0:00:08
                                              0.2/6.7 MB 984.6 kB/s eta 0:00:07
     -                                        0.3/6.7 MB 1.4 MB/s eta 0:00:05
     ---                                      0.5/6.7 MB 2.0 MB/s eta 0:00:04
     ---                                      0.6/6.7 MB 2.1 MB/s eta 0:00:03
     -----                                    1.0/6.7 MB 2.8 MB/s eta 0:00:03
     --------                                 1.4/6.7 MB 3.3 MB/s eta 0:00:02
     --------                                 1.4/6.7 MB 3.5 MB/s eta 0:00:02
     --------                                 1.4/6.7 MB 3.5 MB/s eta 0:00:02
     -----------                              1.9/6.7 MB 3.7 MB/s eta 0:

In [6]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re
import os

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rashp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rashp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
stopwords = set(stopwords.words("english"))

In [13]:
text_dir = r'C:\Users\rashp\Documents\Scrapped Data Files From Given link'
sentment_dir = r'C:\Users\rashp\Documents\MasterDictionary'

In [18]:
# load all text files  from the  directory and store in a list(docs)
docs = []
for text_file in os.listdir(text_dir):
    with open(os.path.join(text_dir,text_file),'r') as f:
        text = f.read()
#tokenize the given text file
        words = word_tokenize(text)
# remove the stop words from the tokens
        filtered_text = [word for word in words if word.lower() not in stopwords]
# add each filtered tokens of each file into a list
        docs.append(filtered_text)
    
text_dir = r'C:\Users\rashp\Documents\Scrapped Data Files From Given link'
sentment_dir = r'C:\Users\rashp\Documents\MasterDictionary'    
    
# store positive, Negative words from the directory
pos=set()
neg=set()

for files in os.listdir(sentment_dir):
    if files =='positive-words.txt':
        with open(os.path.join(sentment_dir,files)) as f:
            pos.update(f.read().splitlines())
    else:
        with open(os.path.join(sentment_dir,files),'r') as f:
            neg.update(f.read().splitlines())

        
# now collect the positive  and negative words from each file
# calculate the scores from the positive and negative words 
positive_words     = []
Negative_words     = []
positive_score     = []
negative_score     = []
polarity_score     = []
subjectivity_score = []



#iterate through the list of docs
for i in range(len(docs)):
      positive_words.append([word for word in docs[i] if word.lower() in pos])
      Negative_words.append([word for word in docs[i] if word.lower() in neg])
      positive_score.append(len(positive_words[i]))
      negative_score.append(len(Negative_words[i]))
      polarity_score.append((positive_score[i] - negative_score[i]) / ((positive_score[i] + negative_score[i]) + 0.000001))
      subjectivity_score.append((positive_score[i] + negative_score[i]) / ((len(docs[i])) + 0.000001))
    
    
    
# Average Sentence Length = the number of words / the number of sentences
# Percentage of Complex words = the number of complex words / the number of words 
# Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

avg_sentence_length          = []
Percentage_of_Complex_words  = []
Fog_Index                    = []
complex_word_count           = []
avg_syllable_word_count      = []

#stopwords = set(stopwords.words('english'))
def measure(file):
      with open(os.path.join(text_dir, file),'r') as f:
        text = f.read()
# remove punctuations 
        text = re.sub(r'[^\w\s.]','',text)
# split the given text file into sentences
        sentences = text.split('.')
# total number of sentences in a file
        num_sentences = len(sentences)
# total words in the file
        words = [word  for word in text.split() if word.lower() not in stopwords ]
        num_words = len(words)
 
# complex words having syllable count is greater than 2
# Complex words are words in the text that contain more than two syllables.
        complex_words = []
        for word in words:
            vowels = 'aeiou'
            syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
            if syllable_count_word > 2:
                complex_words.append(word)

# Syllable Count Per Word
# We count the number of Syllables in each word of the text by counting the vowels present in each word.
#  We also handle some exceptions like words ending with "es","ed" by not counting them as a syllable.
        syllable_count = 0
        syllable_words =[]
        for word in words:
            if word.endswith('es'):
                word = word[:-2]
            elif word.endswith('ed'):
                word = word[:-2]
            vowels = 'aeiou'
            syllable_count_word = sum( 1 for letter in word if letter.lower() in vowels)
            if syllable_count_word >= 1:
                syllable_words.append(word)
                syllable_count += syllable_count_word


        avg_sentence_len = num_words / num_sentences
        avg_syllable_word_count = syllable_count / len(syllable_words)
        Percent_Complex_words  =  len(complex_words) / num_words
        Fog_Index = 0.4 * (avg_sentence_len + Percent_Complex_words)

        return avg_sentence_len, Percent_Complex_words, Fog_Index, len(complex_words),avg_syllable_word_count

# iterate through each file or doc
for file in os.listdir(text_dir):
      x,y,z,a,b = measure(file)
      avg_sentence_length.append(x)
      Percentage_of_Complex_words.append(y)
      Fog_Index.append(z)
      complex_word_count.append(a)
      avg_syllable_word_count.append(b)
    
def cleaned_words(file):
    with open(os.path.join(text_dir,file), 'r') as f:
        text = f.read()
        text = re.sub(r'[^\w\s]', '' , text)
        words = [word  for word in text.split() if word.lower() not in stopwords]
        length = sum(len(word) for word in words)
        average_word_length = length / len(words)
    return len(words),average_word_length

word_count = []
average_word_length = []
for file in os.listdir(text_dir):
      x, y = cleaned_words(file)
      word_count.append(x)
      average_word_length.append(y)
# To calculate Personal Pronouns mentioned in the text, we use regex to find 
# the counts of the words - “I,” “we,” “my,” “ours,” and “us”. Special care is taken
#  so that the country name US is not included in the list.
def count_personal_pronouns(file):
    with open(os.path.join(text_dir,file), 'r') as f:
        text = f.read()
        personal_pronouns = ["I", "we", "my", "ours", "us"]
        count = 0
        for pronoun in personal_pronouns:
            count += len(re.findall(r"\b" + pronoun + r"\b", text)) # \b is used to match word boundaries
    return count

pp_count = []

for file in os.listdir(text_dir):
    x = count_personal_pronouns(file)
    pp_count.append(x)

output_df = pd.read_excel('Output Data Structure.xlsx')

# URL_ID 44 ,57, 144 does not exists i,e. page does not exist, throughs 404 error
# so we are going to drop these rows from the table
output_df.drop([44-36,57-36,144-36], axis = 0, inplace=True)

# These are the required parameters 
variables = [positive_score,
            negative_score,
            polarity_score,
            subjectivity_score,
            avg_sentence_length,
            Percentage_of_Complex_words,
            Fog_Index,
            avg_sentence_length,
            complex_word_count,
            word_count,
            avg_syllable_word_count,
            pp_count,
            average_word_length]

# write the values to the dataframe
for i, var in enumerate(variables):
    output_df.iloc[:,i+2] = var

#now save the dataframe to the disk
output_df.to_csv('Output_Data_1.csv')

C:\Users\rashp\AppData\Local\Temp\ipykernel_7796\4210619926.py:173: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  output_df.iloc[:,i+2] = var
